# Analysis DNS consumption with multiple configurations

Configurations have different QPS and protocols used.
This notebook aims to visualize the difference of consumption between those configurations.
Later we will use models to predict the agregated consumption of those different configurations.
In those configurations, the cache is *always used* because the domain is always the same in the requests (example.com).

## What will visualisations will be shown

The different things we want to study are:
- First an analysis on queries:
    - Idle consumption
    - QPS for different protocols received/sent by DNS and PC
    - The agregated consumption in function of QPS (Query Per Second)
    - The agregated consumption in function of the protocol 
- Then analysis on PC metrics:
    - Visualisation of the metrics for different protocols and different QPS
    - Correlation matrix of all metrics with protocols and QPS in the metrics
- Visualisation of the predicted aggregated consumption made by different models

## The model training part

Different models will be train to predict the aggregated consumption of the DNS server (Random forest, XGBoost, Linear regression and LSTM).
The error will be measured using RMSE (Root Mean Square Error).
A MAPE (Mean Absolute Percentage Error) will give us the error % for all models trained.

## Data transformations

Different type of files are used for this analysis (.log and .csv). They will all be imported using Pandas creating DataFrames.
Here are all the files used:
- CSV files:
    - `cpu_power.csv` : Metrics for the cpu consumption
    - `io_power.csv` : Metrics for the I/O (Disk)
    - `nic_power.csv` : Metrics for the NIC (Network Interface Card)
    - `ram_power.csv` : Metrics fo the ram utilisation (only LLC-loads and LLC-store, Last Level Cache not really RAM but we can estimate ram consumption from it)
    - `yoctowatt.csv` : External measurements made with a YoctoPuce
- Log files:
    - `bind_queries_captired.log` : Queries captured by the Bind DNS
    - `dns_doh.log` : DoH queries captured by the server
    - `dns_tcp.log` : DoT queries captured by the server
    - `dns_udp.log` : UDP queries captured by the server

Log files are used for their queries received at the datetime given.
CSV files contain TimeSeries metrics.

All DataFrames are linked with Date time.

#### NaN gestion

Because all measures don't have the same frequency (100Hz for the YoctoPuce and 1Hz for the rest) and are not exactly synchronised, NaN values will be created when merging them.
NaN values will be interpolated for all measures metrics with `bfill` and NaN from logs will be set to 0.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [16]:
CONFIGURATIONS = {
    "QPS": [1, 10, 100, 500, 1000],
    "Protocols": ["udp", "dot", "doh"]
}

DURATION = 300 # seconds
ROOT = "../data/300_config_merged/"

## We need to collect all files from all those configurations

A function will help collect all files from one configuration.  
All configurations are stored in folders with name `{DURATION}s_{PROTOCOL}_{QPS}qps_{TIMESTAMP}`.  
To simplify this, folders will be collected ignoring the timestamp just with the begining of the folder name.  

#### Function to import and process log files

In [19]:
import os

def import_log_file(file_path, offset_seconds=0, offset_date=None):
    with open(file_path, "r", encoding="utf-8", errors="replace") as f:
        log_df = pd.DataFrame({"raw": f.read().splitlines()})
        
    pattern = r'^(?P<date>\d{2}-[A-Za-z]{3}-\d{4})\s+(?P<time>\d{2}:\d{2}:\d{2}\.\d+).*$'
    if offset_date:
        pattern = r'^(?P<time>\d{2}:\d{2}:\d{2}\.\d+).*$'
    
    log_df = log_df["raw"].str.extract(pattern)
    
    if offset_date:
        log_df["date"] = offset_date.strftime("%d-%b-%Y")

    log_df["datetime"] = pd.to_datetime(log_df["date"] + " " + log_df["time"], format="%d-%b-%Y %H:%M:%S.%f")
    log_df["datetime"] = log_df["datetime"] + pd.to_timedelta(-offset_seconds, unit='s')
    log_df = log_df.drop(columns=["date", "time"])

    log_df.dropna(inplace=True)
    if offset_date is None:
        date_offset = log_df["datetime"].iloc[0].date()
    
    log_df = log_df.set_index("datetime").sort_index()
    
    if offset_date:
        return log_df
    return log_df,date_offset

def import_all_logs(folder_path):
    log_files = [
        "bind_queries_captured.log",
        "dns_doh.log",
        "dns_tcp.log",
        "dns_udp.log",
    ]

    dfs = []
    for log_file in log_files:
        file_path = os.path.join(folder_path, "logs", log_file)
        if "captured" in log_file:
            df, date_offset = import_log_file(file_path)
        else:
            df = import_log_file(file_path, offset_seconds=3600, offset_date=date_offset)
        dfs.append(df)
        
    return dfs

#### Function to import and process the 5 CSV files

In [4]:
def import_csv_files(folder_path):
    # Importing cpu_power.csv
    cpu_power = pd.read_csv(f"{folder_path}/measures/cpu_power.csv")
    # convert Time_Of_Day_Seconds to int
    cpu_power["datetime"] = pd.to_datetime(cpu_power["Time_Of_Day_Seconds"], unit='s')
    cpu_power.drop(columns=["Time_Of_Day_Seconds"], inplace=True)

    # Importing io_power.csv
    io_power = pd.read_csv(f"{folder_path}/measures/io_power.csv")
    io_power["datetime"] = pd.to_datetime(io_power["timestamp"], unit='s')
    io_power.drop(columns=["timestamp"], inplace=True)

    # Importing nic_power.csv
    nic_power = pd.read_csv(f"{folder_path}/measures/nic_power.csv")
    nic_power["datetime"] = pd.to_datetime(nic_power["timestamp"], unit='s')
    nic_power.drop(columns=["timestamp"], inplace=True)

    # Importing ram_power.csv
    ram_power = pd.read_csv(f"{folder_path}/measures/ram_power.csv")
    ram_power["datetime"] = pd.to_datetime(ram_power["timestamp"], unit='s')
    ram_power.drop(columns=["timestamp"], inplace=True)

    # Importing yoctowatt.csv
    yoctowatt = pd.read_csv(f"{folder_path}/measures/yoctowatt.csv")
    yoctowatt["datetime"] = pd.to_datetime(yoctowatt["timestamp_unix"], unit='s')
    yoctowatt = yoctowatt.drop(columns=["reportFrequency_effective", "timestamp_iso", "timestamp_unix"])

    return [cpu_power, io_power, nic_power, ram_power, yoctowatt]

In [ ]:
def collect_df_from_config(QPS, protocol):
    folder_name_start = f"{DURATION}s_{protocol}_{QPS}qps_"
    folder_name = next((f for f in os.listdir(ROOT) if f.startswith(folder_name_start)), None)
    
    logs_dfs = import_all_logs(os.path.join(ROOT, folder_name))
    measures_dfs = import_csv_files(os.path.join(ROOT, folder_name))

    return {
        "logs": logs_dfs,
        "measures": measures_dfs,
        "QPS": QPS,
        "Protocol": protocol
    }

In [30]:
def collect_idle_df():
    folder_name_start = f"idle_"
    folder_name = next((f for f in os.listdir(ROOT) if f.startswith(folder_name_start)), None)
    
    logs_dfs = import_all_logs(os.path.join(ROOT, folder_name))
    measures_dfs = import_csv_files(os.path.join(ROOT, folder_name))

    return {
        "logs": logs_dfs,
        "measures": measures_dfs,
        "QPS": 0,
        "Protocol": "idle"
    }

#### Collect all DataFrames from all configurations

In [29]:
configs = []
for protocol in CONFIGURATIONS["Protocols"]:
    for qps in CONFIGURATIONS["QPS"]:
        configs.append(collect_df_from_config(qps, protocol))
idle_config = collect_idle_df()
configs.append(idle_config)

IndexError: single positional indexer is out-of-bounds

In [33]:
# idle_config = collect_idle_df()
folder_name_start = f"idle_"
folder_name = next((f for f in os.listdir(ROOT) if f.startswith(folder_name_start)), None)
folder_name


'idle_20260217_084146'